# Scripps Automated Shore Station (SASS) Self-Calibrating SeapHOx (SCS) live data viewer
### This script scrubs .dat files from the SCCOOS dr: https://sccoos.org/dr/data/. The SCS measures pH, oxygen, salinity and temperature. The pH values plotted here are calibrated using an average of the coefficients established pre-deployment and from the  automated tris calibrations. Oxygen concentration has been corrected for salinity and pressure. Before using and sharing these values, please get permission from the author. A new tab delimited text file will be produced every time the script runs, with the current timestamp of the time the file was created. All time zones associated with this script are in UTC.
### Created by: Taylor Wirth twirth@ucsd.edu
### Last edit: 18 March 2022

In [194]:
# load the .py scripts
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import glob
%load_ext autoreload
%autoreload 1
%aimport get_recent_dr
%aimport get_all_dr
%aimport correct_DO_with_sal
%aimport pHtris_from_T
%aimport k0int_from_Vint_pHcal
%aimport k0ext_from_Vext_pHcal
%aimport pHint_from_Vint_k0int
%aimport pHext_from_Vext_k0ext
%aimport plot_all

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The last hour of data and plot of all the data will show below. Please wait!

In [198]:
# Get all the files for the Scripps Pier SCS. get_all_dr.py looks and grabs all data files for the Scripps Pier SCS, 
# then converts it into a pandas DataFrame. 
# Printed below is the last hour of data. This may take a few seconds depending on last data download!
path = 'https://sccoos.org/dr/data/scripps_pier_scs/'

# if no txt file of downloaded data, get all of the data:
txt = glob.glob('*.txt')
if txt:
    df, col_names = get_recent_dr.get_recent_dr(path)
    # delete current txt file
    os.remove(txt[0])
else:
    df, col_names = get_all_dr.get_all_dr(path)

# create new tab delimited file to download
from datetime import datetime
now = datetime.utcnow()
current_time = now.strftime('%Y%m%d%H%M%S')
df.to_csv('SIOpierSCS_download_'+current_time+'.txt', header=col_names, index=None, sep='\t')

df.tail(6)

,internet_datetime,IP,samp_type,samp_num,calib_num,calib_rep,date,time,vbatt,vtherm,...,Bpot,Ramp,Raw_Temp,SBEtemp,SBEcond,SBEsal,SBEday,SBEmon,SBEyear,SBEtime
3954,2022-03-18T23:10:28Z,172.16.117.226,0,4060,2,0,2022/03/18,23:10:10,15.53,0.53906,...,1155.2,642.9,306.2,16.9195,4.31025,33.5043,18,Mar,2022,15:24:16
3955,2022-03-18T23:20:28Z,172.16.117.226,0,4061,2,0,2022/03/18,23:20:10,15.53,0.54008,...,1149.1,642.3,308.0,16.7738,4.29405,33.4849,18,Mar,2022,15:34:16
3956,2022-03-18T23:30:28Z,172.16.117.226,0,4062,2,0,2022/03/18,23:30:10,15.53,0.53830,...,1145.2,642.0,304.0,16.9185,4.30920,33.4960,18,Mar,2022,15:44:16
3957,2022-03-18T23:40:28Z,172.16.117.226,0,4063,2,0,2022/03/18,23:40:10,15.53,0.53334,...,1142.5,641.4,293.4,17.4631,4.36064,33.4883,18,Mar,2022,15:54:16
3958,2022-03-18T23:50:28Z,172.16.117.226,0,4064,2,0,2022/03/18,23:50:10,15.53,0.52840,...,1140.5,640.2,284.4,17.8612,4.39947,33.4923,18,Mar,2022,16:04:16
3959,2022-03-19T00:00:28Z,172.16.117.226,0,4065,2,0,2022/03/19,00:00:10,15.53,0.52487,...,1138.9,638.6,278.4,17.9921,4.41120,33.4847,18,Mar,2022,16:14:16


In [199]:
# Oxygen correction for salinity and pressure
O2_corr = correct_DO_with_sal.correct_DO_with_sal(df.O2con, df.O2temp, df.press, df.SBEsal, sal_input=0)

In [203]:
# run this only once!!
caldf = df.loc[df['samp_type']==1] # calibration samples
df = df.drop(df[df.samp_type==1].index) # remove calibration samples from the data frame

In [219]:
# SeapHOx calibration from automated tris measurements
pHtris = pHtris_from_T.pHtris_from_T(caldf.SBEtemp, S=35)
k0int_tris = k0int_from_Vint_pHcal.k0int_from_Vint_pHcal(caldf.vint, pHtris, caldf.SBEtemp)
k0ext_tris = k0ext_from_Vext_pHcal.k0ext_from_Vext_pHcal(caldf.vext, pHtris, caldf.SBEtemp, calsal=35)

# tank calib, and first 2 calibrations
k0int_tris_before = pd.Series([-0.3961671, -0.3971597, -0.3969670])
k0int_tris_all = k0int_tris_before.append(k0int_tris,ignore_index=True)
k0ext_tris_before = pd.Series([-1.3843647, -1.3849238,  -1.3847827])
k0ext_tris_all = k0ext_tris_before.append(k0ext_tris,ignore_index=True)
tris_date_before = pd.Series(['2022/01/18','2022/02/18','2022/02/18'])
tris_date_all = tris_date_before.append(caldf.date,ignore_index=True)

caldatetime = list(caldf['date'] + ' ' + caldf['time']) # combine date and time to use for plots
dfk0 = pd.DataFrame({'caldate':tris_date_all,'k0int_tris':k0int_tris_all, 'k0ext_tris':k0ext_tris_all})
print(dfk0)
k0int_mean = dfk0.mean(axis=0)[0]
k0ext_mean = dfk0.mean(axis=0)[1]
print('k0int_mean = ',k0int_mean)
print('k0ext_mean = ',k0ext_mean)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=dfk0.caldate, y=dfk0.k0int_tris,mode='markers',name='k0int_tris',marker_symbol='star',marker_size=15,marker_color='blue'),secondary_y=False)
fig.add_trace(go.Scatter(x=dfk0.caldate, y=dfk0.k0ext_tris,mode='markers',name='k0ext_tris',marker_symbol='square',marker_size=10,marker_color='red'),secondary_y=True)
fig.add_hline(y=k0int_mean,secondary_y=False,name='k0int_mean',line_color='blue',annotation_text="k0 means",annotation_position="top left") # mean k0int for all calibrations
fig.add_hline(y=k0ext_mean,secondary_y=True,name='k0int_mean',line_color='red') # mean k0ext for all calibrations
fig.update_layout(height=500, width=950,title_text='Calibration coefficients at 0C')
fig['layout']['yaxis']['title']='k0_int'
fig['layout']['yaxis2']['title']='k0_ext'
fig.show()

      caldate  k0int_tris  k0ext_tris
0  2022/01/18   -0.396167   -1.384365
1  2022/02/18   -0.397160   -1.384924
2  2022/02/18   -0.396967   -1.384783
3  2022/03/04   -0.396616   -1.384933
4  2022/03/04   -0.396444   -1.384783
5  2022/03/18   -0.396388   -1.384749
6  2022/03/18   -0.396321   -1.384647
k0int_mean =  -0.3965803454136096
k0ext_mean =  -1.3847405207112065


In [205]:
datetime = list(df['date'] + ' ' + df['time']) # combine date and time to use for plots
Vint = df.vint.astype(float)
Vext = df.vext.astype(float)
T_C = df.SBEtemp.astype(float)
sal = df.SBEsal.astype(float)
pHint_SBE = pHint_from_Vint_k0int.pHint_from_Vint_k0int(k0int_mean, Vint, T_C)
pHint_tris = pHint_from_Vint_k0int.pHint_from_Vint_k0int(k0int_tris,Vint, T_C)
pHext_SBE = pHext_from_Vext_k0ext.pHext_from_Vext_k0ext(k0ext_mean, Vext, T_C, sal)
pHext_tris = pHext_from_Vext_k0ext.pHext_from_Vext_k0ext(k0ext_tris, Vext, T_C, sal)

In [233]:
plot_all.plot_all(datetime,pHint_SBE,pHext_SBE,O2_corr,df.O2sat,df.SBEtemp,df.O2temp,df.pHtemp,df.SBEsal,df.press)